In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric import utils
import networkx as nx
import pandas as pd
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pygraphviz as pgv
import numpy as np
from node2vec import Node2Vec
import pickle
import matplotlib.pyplot as plt
from PIL import Image
# Image.MAX_IMAGE_PIXELS = 3075432330


# In[5]:

In [2]:
graph_source = pd.read_csv('to_graph.csv')
df = pd.read_csv('final_result_mibig.csv')


# In[13]:


graph_source


# In[24]:


node_organism = list(graph_source['organism_name'])
node_organism_colored = []

node_compound = list(graph_source['compound'])
node_compound_colored = []

edges = []
for k in range(len(node_organism)):
    edges.append((node_organism[k], node_compound[k]))


# In[27]:
# organism ---> compound
pos = {}
#pos.update((node, (0, index)) for index, node in enumerate(node_organism))
#pos.update((node, (1, index)) for index, node in enumerate(node_compound))
G = nx.DiGraph()
G.add_nodes_from(node_organism, bipartite=0, color='red')
G.add_nodes_from(node_compound, bipartite=1, color='green')
G.add_edges_from(edges)
print(len(node_compound))
print(len(node_organism))

2689
2689


In [3]:
print(len(G.nodes))
print(G.edges)

3521
[('Verrucosispora maris AB-18-032', 'abyssomicin C'), ('Verrucosispora maris AB-18-032', 'atrop-abyssomicin C'), ('Kutzneria albida DSM 43870', 'aculeximycin'), ('Alternaria alternata', 'AF-toxin'), ('Alternaria alternata', 'dimethylcoprogen'), ('Alternaria alternata', 'UNII-YC2Q1O94PT'), ('Alternaria alternata', 'ACT-Toxin II'), ('Alternaria alternata', 'AM-toxin'), ('Alternaria alternata', 'AK-toxin I'), ('Aspergillus oryzae', 'aflatoxin G1'), ('Aspergillus oryzae', 'ferrichrome'), ('Aspergillus oryzae', 'cyclopiazonic acid'), ('Aspergillus parasiticus', 'aflatoxin'), ('Aspergillus flavus', 'aflatoxin'), ('Aspergillus flavus', 'cyclopiazonic acid'), ('Aspergillus nomius', 'aflatoxin'), ('Aspergillus ochraceoroseus', 'sterigmatocystin'), ('Alternaria solani', 'alternapyrone'), ('Alternaria solani', 'solanapyrone D'), ('Aspergillus nidulans FGSC A4', 'alternariol'), ('Aspergillus nidulans FGSC A4', 'asperfuranone'), ('Aspergillus nidulans FGSC A4', 'cichorine'), ('Aspergillus nidu

In [4]:
# applied Graph convolution network to learn features
if torch.cuda.is_available():
    device = 'cuda'
print('use'+device)
def representation_learning(input_dim, hidden_dim, output_dim, data):
    dropout = 0.5
    convs = nn.ModuleList()
    convs.append(GCNConv(input_dim, hidden_dim))
    num_layers = 3
    heads = 1
    for l in range(num_layers-1):
        convs.append(GCNConv(heads * hidden_dim, hidden_dim))

    post_mp = nn.Sequential(
        nn.Linear(heads * hidden_dim, hidden_dim), nn.Dropout(dropout),
        nn.Linear(hidden_dim, output_dim))


    x, edge_index, batch = data.x, data.edge_index, data.batch

    for i in range(num_layers):
        x = convs[i](x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=dropout)

    x = post_mp(x)
    return x, F.log_softmax(x, dim=1)


usecuda


In [5]:
G_1 = nx.convert_node_labels_to_integers(G)

In [6]:
node_organism_label = np.ones([1, len(node_organism) ])
node_compound_label = np.zeros([1, len(node_compound) ])
print('compound length: ', len(node_compound))
print('organism length', len(node_organism))


Data1 = utils.from_networkx(G_1)
# node_compound.append(node_organism)
x = torch.tensor([node_compound_label, node_organism_label], dtype=torch.float)
print(x.shape)
print(x)
x =x.view(-1, 5378).t()
print(x.shape)
print(x)
edges_list = list(G_1.edges)
edge_index = torch.tensor(edges_list, dtype=torch.long)
data = Data(x=x, edge_index=edge_index.t().contiguous())

hidden_dim = 128
data.edge_index
print(data)
print(data.num_node_features)
a, b = representation_learning(data.num_node_features, hidden_dim, 128, data)

compound length:  2689
organism length 2689
torch.Size([2, 1, 2689])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]]])
torch.Size([5378, 1])
tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])
Data(x=[5378, 1], edge_index=[2, 2581])
1


C:\Users\asse9\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  if __name__ == '__main__':


In [9]:
data.x

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])

In [85]:
a = a.to('cpu').detach().numpy().copy()
b = b.to('cpu').detach().numpy().copy()

AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [92]:
print(a.shape)
print(b.shape)
print(a[0].max(), a[0].min())
print(b[0].max(), b[0].min())
a_compound = a[0:2689]
print(a_compound.shape)
np.save('o2c_gcn.npy', a_compound)

(5378, 128)
(5378, 128)
0.176171 -0.12135337
-4.6834326 -4.980957
(2689, 128)


In [78]:
from node2vec import Node2Vec

node2vec = Node2Vec(G)
# Embed
model = node2vec.fit()

Computing transition probabilities:   0%|          | 0/3521 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 21.62it/s]


In [89]:
# convert node2vec to dataframe
emb_df = (
    pd.DataFrame(
        [model.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)
emb_df.shape
print(emb_df[0].max, emb_df[0].min)

<bound method NDFrame._add_numeric_operations.<locals>.max of Verrucosispora maris AB-18-032    0.001180
Kutzneria albida DSM 43870        0.001247
Alternaria alternata             -0.002864
Aspergillus oryzae                0.006084
Aspergillus parasiticus           0.005850
                                    ...   
perquinoline A                    0.004823
perquinoline B                   -0.000784
perquinoline C                   -0.003117
ilicicolin H                      0.001977
dehydrofosmidomycin              -0.000580
Name: 0, Length: 3521, dtype: float64> <bound method NDFrame._add_numeric_operations.<locals>.min of Verrucosispora maris AB-18-032    0.001180
Kutzneria albida DSM 43870        0.001247
Alternaria alternata             -0.002864
Aspergillus oryzae                0.006084
Aspergillus parasiticus           0.005850
                                    ...   
perquinoline A                    0.004823
perquinoline B                   -0.000784
perquinoline C      

In [3]:
## check if the microbe-compound interaction graph is directed graph and the directions are right
## also check number of edges
T_edge = 0
F_edge = 0
a = []
for k, v in enumerate(node_organism):
    a.append((v, node_compound[k]))
    if G.has_edge(v, node_compound[k]) == True:
        T_edge +=1
    if G.has_edge(node_compound[k], v) == True:
        F_edge +=1
print(G.number_of_edges())
print(T_edge)
print(F_edge)

2581
2689
0


In [14]:
seen = set()
dupes = []

for x in a:
    if x in seen:
        dupes.append(x)
    else:
        seen.add(x)

In [16]:
len(dupes)

108